In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = (
    SparkSession.builder.master("local[2]")
    .appName("demo")
    .config(
        map={
            "spark.driver.memory": "2G",
            "spark.jars.packages": "com.datastax.spark:spark-cassandra-connector_2.12:3.4.1,org.postgresql:postgresql:42.6.0",
            "spark.sql.extensions": "com.datastax.spark.connector.CassandraSparkExtensions",
            "spark.sql.catalog.comwatt": "com.datastax.spark.connector.datasource.CassandraCatalog",
            "spark.sql.catalog.comwatt.spark.cassandra.connection.compression": "LZ4",
            "spark.sql.catalog.comwatt.spark.cassandra.connection.host": "localhost:9042",
        }
    )
    .getOrCreate()
)

23/12/11 23:38:39 WARN Utils: Your hostname, LTO-CWT060 resolves to a loopback address: 127.0.1.1; using 192.168.0.23 instead (on interface wlp0s20f3)
23/12/11 23:38:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/chen/.ivy2/cache
The jars for the packages stored in: /home/chen/.ivy2/jars
com.datastax.spark#spark-cassandra-connector_2.12 added as a dependency
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f380e891-7f98-4187-b97d-785db03bd097;1.0
	confs: [default]


:: loading settings :: url = jar:file:/home/chen/Documents/presentation-meetup-datascience/demo/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found com.datastax.spark#spark-cassandra-connector_2.12;3.4.1 in central
	found com.datastax.spark#spark-cassandra-connector-driver_2.12;3.4.1 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.11.0 in central
	found com.datastax.oss#java-driver-core-shaded;4.13.0 in central
	found com.datastax.oss#native-protocol;1.5.0 in central
	found com.datastax.oss#java-driver-shaded-guava;25.1-jre-graal-sub-1 in local-m2-cache
	found com.typesafe#config;1.4.1 in local-m2-cache
	found org.slf4j#slf4j-api;1.7.26 in local-m2-cache
	found io.dropwizard.metrics#metrics-core;4.1.18 in central
	found org.hdrhistogram#HdrHistogram;2.1.12 in local-m2-cache
	found org.reactivestreams#reactive-streams;1.0.3 in central
	found com.github.stephenc.jcip#jcip-annotations;1.0-1 in local-m2-cache
	found com.github.spotbugs#spotbugs-annotations;3.1.12 in local-m2-cache
	found com.google.code.findbugs#jsr305;3.0.2 in spark-list
	found com.datastax.oss#java-driver-mapper-runtime;4.13.0 in centr

In [5]:
from demo.adapters.postgres_reader import PostgresClient, PostgresReader

pg_client = PostgresClient(
    spark, "jdbc:postgresql://localhost:5432/comwatt", "postgres", "demo_password"
)
device_reader = PostgresReader(pg_client, "devices")
device_reader().show(truncate=False)

+----------+------------------------------------+-------+---------+-------------------------+
|device_uid|connected_object_uid                |site_id|type     |label                    |
+----------+------------------------------------+-------+---------+-------------------------+
|0         |05ed9851-5343-47d9-9417-99422184eb29|0      |chauffage|Mon chauffage maison     |
|1         |0fd49130-1973-418f-b101-3682942654e0|1      |chauffage|Le chauffage c'est la vie|
|2         |14766b2f-4d6f-4849-afdf-d74639ac67a4|2      |four     |pour faire du pain       |
+----------+------------------------------------+-------+---------+-------------------------+



In [6]:
from demo.adapters.cassandra_reader import CassandraClient, CassandraReader

cass_client = CassandraClient(spark, "comwatt.comwatt")
measures_reader = CassandraReader(cass_client, "measures")
measures_reader().show(truncate=False)

+-------+-----------+------------------------------------+-------------------+----------+
|site_id|time_bucket|connected_object_uid                |timestamp          |value     |
+-------+-----------+------------------------------------+-------------------+----------+
|0      |202309     |05ed9851-5343-47d9-9417-99422184eb29|2023-09-12 01:00:00|93.01135  |
|0      |202309     |05ed9851-5343-47d9-9417-99422184eb29|2023-09-12 00:00:00|102.2616  |
|0      |202309     |05ed9851-5343-47d9-9417-99422184eb29|2023-09-11 23:00:00|85.77224  |
|0      |202309     |05ed9851-5343-47d9-9417-99422184eb29|2023-09-11 22:00:00|86.07031  |
|0      |202309     |05ed9851-5343-47d9-9417-99422184eb29|2023-09-11 21:00:00|96.67746  |
|0      |202309     |05ed9851-5343-47d9-9417-99422184eb29|2023-09-11 20:00:00|96.364456 |
|0      |202309     |05ed9851-5343-47d9-9417-99422184eb29|2023-09-11 19:00:00|103.131714|
|0      |202309     |05ed9851-5343-47d9-9417-99422184eb29|2023-09-11 18:00:00|103.00402 |
|0      |2

In [7]:
from demo.get_measures_of_date_for_devices import get_measures_of_date_for_devices

get_measures_of_date_for_devices("2023-09-11", device_reader, measures_reader).explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [site_id#146, connected_object_uid#145, device_uid#144, type#147, label#148, time_bucket#135, timestamp#137, value#138]
   +- SortMergeJoin [cast(site_id#146 as bigint), connected_object_uid#145], [site_id#134L, connected_object_uid#136], Inner
      :- Sort [cast(site_id#146 as bigint) ASC NULLS FIRST, connected_object_uid#145 ASC NULLS FIRST], false, 0
      :  +- Exchange hashpartitioning(cast(site_id#146 as bigint), connected_object_uid#145, 200), ENSURE_REQUIREMENTS, [plan_id=65]
      :     +- Scan JDBCRelation(devices) [numPartitions=1] [device_uid#144,connected_object_uid#145,site_id#146,type#147,label#148] PushedFilters: [*IsNotNull(site_id), *IsNotNull(connected_object_uid)], ReadSchema: struct<device_uid:int,connected_object_uid:string,site_id:int,type:string,label:string>
      +- Sort [site_id#134L ASC NULLS FIRST, connected_object_uid#136 ASC NULLS FIRST], false, 0
         +- Exchange hashpartitioning(sit

In [27]:
import pyspark.sql.functions as F

df_device = device_reader()
df_measure = measures_reader()
device_by_site = (
    df_device.groupBy("site_id")
    .agg(F.collect_list("connected_object_uid").alias("uids"))
    .collect()
)

for row in device_by_site:
    (
        df_measure.filter(F.col("site_id") == row.site_id)
        .filter(F.col("time_bucket") == "202309")
        .filter(F.col("connected_object_uid").isin(row.uids))
        .filter(F.col("timestamp") >= "2023-09-11")
        .filter(F.col("timestamp") < "2023-09-12")
        .explain()
    )

== Physical Plan ==
*(1) Project [site_id#918L, time_bucket#919, connected_object_uid#920, timestamp#921, value#922]
+- BatchScan measures[site_id#918L, time_bucket#919, connected_object_uid#920, timestamp#921, value#922] Cassandra Scan: comwatt.measures
 - Cassandra Filters: [["timestamp" < ?, 2023-09-12 00:00:00.0],["connected_object_uid" = ?, 0fd49130-1973-418f-b101-3682942654e0],["timestamp" >= ?, 2023-09-11 00:00:00.0],["site_id" = ?, 1],["time_bucket" = ?, 202309]]
 - Requested Columns: [site_id,time_bucket,connected_object_uid,timestamp,value] RuntimeFilters: []


== Physical Plan ==
*(1) Project [site_id#918L, time_bucket#919, connected_object_uid#920, timestamp#921, value#922]
+- BatchScan measures[site_id#918L, time_bucket#919, connected_object_uid#920, timestamp#921, value#922] Cassandra Scan: comwatt.measures
 - Cassandra Filters: [["timestamp" < ?, 2023-09-12 00:00:00.0],["timestamp" >= ?, 2023-09-11 00:00:00.0],["connected_object_uid" = ?, 14766b2f-4d6f-4849-afdf-d74639ac